In [31]:
from pyTM import *
import itertools

## 1.8.а
$\Sigma = \{a, b, c\}$

Замінити всі входження **a** на **bb**

In [9]:
# States:
q0, q1a, q1b, q1c, q2 = range(5)
# Input/output alphabet:
"abc"
# Additional machine alphabet:
"#x"

def I(s_in): 
    return s_in

tm_a = ( TMFactory()
    .add_rule(q0 , 'b', q0 , 'b', R) # ignore, proceed
    .add_rule(q0 , 'c', q0 , 'c', R) # ignore, proceed
    .add_rule(q0 , 'a', q1b, 'x', L) # leave marker. start copying till left end

    .add_rule(q1a, 'a', q1a, 'a', L) # copy a. paste a.
    .add_rule(q1b, 'a', q1a, 'b', L) # copy a. paste b.
    .add_rule(q1c, 'a', q1a, 'c', L) # copy a. paste c.

    .add_rule(q1a, 'b', q1b, 'a', L) # copy b. paste a.
    .add_rule(q1b, 'b', q1b, 'b', L) # copy b. paste b.
    .add_rule(q1c, 'b', q1b, 'c', L) # copy b. paste c.

    .add_rule(q1a, 'c', q1c, 'a', L) # copy c. paste a.
    .add_rule(q1b, 'c', q1c, 'b', L) # copy c. paste b.
    .add_rule(q1c, 'c', q1c, 'c', L) # copy c. paste c.

    .add_rule(q1a, '#', q2 , 'a', R) # left end reached. paste a. start returning to marker 
    .add_rule(q1b, '#', q2 , 'b', R) # left end reached. paste a. start returning to marker 
    .add_rule(q1c, '#', q2 , 'c', R) # left end reached. paste a. start returning to marker 

    .add_rule(q2 , 'x', q0 , 'b', R) # marker reached. set it 'b'. proceed.
    .add_rule(q2 , ANY, q2 ,  I , R) # any symbol except marker. do not change it. procceed.
    .build(q0, []) 
 )

In [10]:
for i in ['', 'a', 'b', 'c' 'ab', 'abc', 'bca', 'abcba', 'bcacb', 'bbbcacacacbbb']:
    print(i, "->", tm_a.run(i, debug=False)) 

 -> ('', 1)
a -> ('bb', 4)
b -> ('b', 2)
cab -> ('cbbb', 8)
abc -> ('bbbc', 6)
bca -> ('bcbb', 10)
abcba -> ('bbbcbbb', 20)
bcacb -> ('bcbbcb', 12)
bbbcacacacbbb -> ('bbbcbbcbbcbbcbbb', 62)


## 1.8.б
$\Sigma = \{a, b, c\}$

Замінити всі входження **ab** на **c**

In [11]:
# States:
q0, q1, q2, q3, q3a, q3b, q3c = range(7)
# Input/output alphabet:
"abc"
# Additional machine alphabet:
"#x"

def I(s_in): 
    return s_in

tm_b = ( TMFactory()
    .add_rule(q0 , 'b', q0 , 'b', R) # ignore, proceed
    .add_rule(q0 , 'c', q0 , 'c', R) # ignore, proceed
    .add_rule(q0 , 'a', q1 , 'a', R) # Possible "ab"
        
    .add_rule(q1 , 'a', q1 , 'a', R) # Possible "ab" again, but not yet
    .add_rule(q1 , 'c', q0 , 'c', R) # ignore, proceed
    .add_rule(q1 , 'b', q2 , 'x', L) # "ab" found! leave marker. go to leftmost character.
        
    .add_rule(q2 , '#', q3 , '#', R) # leftmost found. start deleting.
    .add_rule(q2 , ANY, q2 ,  I , L) # nothing interesting. proceed moving left.
        
    .add_rule(q3 , 'a', q3a, '#', R) # copy a. paste #
    .add_rule(q3 , 'b', q3b, '#', R) # copy b. paste #
    .add_rule(q3 , 'c', q3c, '#', R) # copy c. paste #
        
    .add_rule(q3a, 'a', q3a, 'a', R) # copy a. paste a
    .add_rule(q3a, 'b', q3b, 'a', R) # copy b. paste a
    .add_rule(q3a, 'c', q3c, 'a', R) # copy c. paste a
        
    .add_rule(q3b, 'a', q3a, 'b', R) # copy a. paste b
    .add_rule(q3b, 'b', q3b, 'b', R) # copy b. paste b
    .add_rule(q3b, 'c', q3c, 'b', R) # copy c. paste b
        
    .add_rule(q3c, 'a', q3a, 'c', R) # copy a. paste c
    .add_rule(q3c, 'b', q3b, 'c', R) # copy b. paste c
    .add_rule(q3c, 'c', q3c, 'c', R) # copy c. paste c
        
#     .add_rule(q3 , 'x', q0 , '#', R) # marker found. paste #. proceed
    .add_rule(q3a, 'x', q0 , 'c', R) # marker found. paste c. proceed
    .add_rule(q3b, 'x', q0 , 'c', R) # marker found. paste c. proceed
    .add_rule(q3c, 'x', q0 , 'c', R) # marker found. paste c. proceed
        
    .build(q0, []) 
 )

In [12]:
for i in ['', 'a', 'b', 'c', 'ab', 'bc', 'ac', 'abc', 'ba', 'ca', 'cb', 'cabcacbcabab', 'aaa'+'ab'*5+'bbab', 'aabb']:
    print(i, "->", tm_b.run(i, debug=False)) 

 -> ('', 1)
a -> ('a', 2)
b -> ('b', 2)
c -> ('c', 2)
ab -> ('c', 7)
bc -> ('bc', 3)
ac -> ('ac', 3)
abc -> ('cc', 8)
ba -> ('ba', 3)
ca -> ('ca', 3)
cb -> ('cb', 3)
cabcacbcabab -> ('cccacbccc', 57)
aaaabababababbbab -> ('aaacccccbbc', 112)
aabb -> ('acb', 11)


## 1.8.в
$\Sigma = \{a, b\}$

Подвоїти все слово

In [13]:
# States:
q0, q1a, q1b, q2a, q2b, q3, q4, q5 = range(8)
# Input/output alphabet:
"ab"
# Additional machine alphabet:
"#xpqrs"

def I(s_in): 
    return s_in

tm_c = ( TMFactory()
    .add_rule(q0 , 'a', q1a, 'p', R) # save a. put left 'a' marker. go to the end first time.
    .add_rule(q0 , 'b', q1b, 'q', R) # save b. put left 'b' marker. go to the end first time.
    .add_rule(q1a, '#', q3 , 'r', L) # put right 'a' marker. proceed.
    .add_rule(q1b, '#', q3 , 's', L) # put right 'b' marker. proceed.
    .add_rule(q1a, ANY, q1a,  I , R) # ignore. going to the end first time
    .add_rule(q1b, ANY, q1b,  I , R) # ignore. going to the end first time
        
    .add_rule(q2a, '#', q3 , 'a', L) # paste 'a'. start going back to the left marker
    .add_rule(q2b, '#', q3 , 'b', L) # paste 'b'. start going back to the left marker
    .add_rule(q2a, ANY, q2a,  I , R) # ignore. going to the end
    .add_rule(q2b, ANY, q2b,  I , R) # ignore. going to the end
        
    .add_rule(q3 , 'p', q4 , 'a', R) # put 'a' back. go to the next uncopied symbol 
    .add_rule(q3 , 'q', q4 , 'b', R) # put 'b' back. go to the next uncopied symbol 
    .add_rule(q3 , ANY, q3 ,  I , L) # ignore. going to the left marker.
        
    .add_rule(q4 , 'a', q2a, 'p', R) # replace 'a' with left 'a' marker. start going to the end
    .add_rule(q4 , 'b', q2b, 'q', R) # replace 'b' with left 'b' marker. start going to the end
    .add_rule(q4 , 'r', q5 , 'a', S) # right 'a' marker reached. replace with 'a'. finish.
    .add_rule(q4 , 's', q5 , 'b', S) # right 'b' marker reached. replace with 'b'. finish.

    .build(q0, []) 
 )

In [15]:
passed = True
for i in ['', 'a', 'b', 'aaa', 'ababab', 'bbaa', 'abbababbaaababba', 'bab', 'aba', 'abab', 'baba', 'aaabaaa', 'bbbabbb']:
    ans, steps = tm_c.run(i, debug=False)
    if ans != i*2:
        print("Error!",i," -> ",ans,steps)
        passed = False
if passed:
    print("passed")

passed


## 1.8.г
$\Sigma = \{a, b\}$

$x \to x^R$

In [18]:
# States:
q0, q1a, q1b, q2, q3 = range(5)
# Input/output alphabet:
"ab"
# Additional machine alphabet:
"#x"

def I(s_in): 
    return s_in

tm_d = ( TMFactory()
    .add_rule(q0 , 'a', q1a, 'x', L) 
    .add_rule(q0 , 'b', q1b, 'x', L)
    .add_rule(q1a, '#', q2 , 'a', R) 
    .add_rule(q1b, '#', q2 , 'b', R) 
    .add_rule(q1a, ANY, q1a,  I , L) 
    .add_rule(q1b, ANY, q1b,  I , L) 
    .add_rule(q2 , 'x', q0 , 'x', R) 
    .add_rule(q2 , ANY, q2 ,  I , R) 
    .add_rule(q0 , 'x', q0 , 'x', R) 
    .add_rule(q0 , '#', q3 , '#', L) 
    .add_rule(q3 , 'x', q3 , '#', L)
    .build(q0, []) 
 )

In [29]:
tm_d.run('', debug=True)

# : step 0
^ (0)
## : step 1
^ (4)
Finished in 2 steps. 
  []


('', 2)

## 1.8.д
$\Sigma = \{0, 1, 2\}$

Вхід: <двійкове число>2<двійкове число>

Вихід: сума двох чисел в двійковому записі

In [69]:
# States:
q0, q00, q1, q2, s00, s01, s02, s10, s11, s12, s20, s21, s22, e00, e01, e02, e12, f0, f1, f2, f3 = \
"q0, q00, q1, q2, s00, s01, s02, s10, s11, s12, s20, s21, s22, e00, e01, e02, e12, f0, f1, f2, f3".split(", ")

# Input/output alphabet:
"012"
# Additional machine alphabet:
"#x3+"

def I(s_in): 
    return s_in

#x 10010110 2 101010110#

tm_c = ( TMFactory()
    .add_rule(q0 , ANY, q00,  I , L) 
    .add_rule(q00, ANY, q1 , '3', R)
        
    .add_rule(q1 , '#', q2 , '#', L) 
    .add_rule(q1 , ANY, q1 ,  I , R) 
    .add_rule(q2 , 'x', q2 , 'x', L) 
    .add_rule(q2 , '0', s00, 'x', L) 
    .add_rule(q2 , '1', s01, 'x', L) 
    .add_rule(q2 , '2', f0 , '2', R) 
    .add_rule(s00, '2', s10, '2', L) 
    .add_rule(s01, '2', s11, '2', L) 
    .add_rule(s00, ANY, s00,  I , L) 
    .add_rule(s01, ANY, s01,  I , L) 
    .add_rule(s02, ANY, s02,  I , L) 
        
    .add_rule(s10, 'x', s10, 'x', L) 
    .add_rule(s11, 'x', s11, 'x', L) 
    .add_rule(s10, '0', s20, 'x', L)
    .add_rule(s10, '1', s21, 'x', L) 
    .add_rule(s11, '0', s21, 'x', L)
    .add_rule(s11, '1', s22, 'x', L)
    .add_rule(s11, '3', s22, 'x', L)
        
    .add_rule(s20, '3', e00, '3', L)
    .add_rule(s21, '3', e01, '3', L)
    .add_rule(s22, '3', e02, '3', L)
    .add_rule(s20, ANY, s20,  I , L)
    .add_rule(s21, ANY, s21,  I , L)
    .add_rule(s22, ANY, s22,  I , L)
        
    .add_rule(e00, '#', q1 , '0', R)
    .add_rule(e00, '+', q1 , '1', R)
    .add_rule(e01, '#', q1 , '1', R)
    .add_rule(e01, '+', e12, '0', L)
    .add_rule(e02, '#', e12, '0', L)
    .add_rule(e02, '+', e12, '1', L)
    .add_rule(e12, '#', q1 , '+', R)
    
    .add_rule(e00, ANY, e00,  I , L)
    .add_rule(e01, ANY, e01,  I , L)
    .add_rule(e02, ANY, e02,  I , L)
        
    .add_rule(f0 , '#', f1 , '#', L) 
    .add_rule(f0 , ANY, f0 ,  I , R) 
    .add_rule(f1 , '3', f2 , '#', L) 
    .add_rule(f1 , ANY, f1 , '#', L) 
    .add_rule(f2 , '+', f3 , '1', S) 
    .add_rule(f2 , '#', f3 , '#', S) 
    .add_rule(f2 , ANY, f2 ,  I , L) 
    
        
    .build(q0, []) 
 )

In [74]:
passed = True
nums = [(a,b) for a in range(60) for b in range(60)]
for a, b in nums:
    x = f"{a:b}".rjust(10, '0')
    y = f"{b:b}".rjust(10, '0')
    inpt = x + '2' + y
    outpt = tm_c.run(inpt, debug=False)[0]
    if int(outpt, base=2) != a+b:
        print("error:",inpt,"->",outpt)
        passed = False
passed

True